 # Inserting and Querying Data in PostgreSQL

Importing Libraries

In [1]:
# pip install psycopg2-binary
import pandas as pd
import psycopg2


Loading the datasets

In [7]:
# Read CSV files into pandas DataFrames
df_banks = pd.read_csv("/Users/elbethelzewdie/Documents/customers-satisfaction-week2/customers-satisfaction-week2/data/bank.csv")
df_reviews = pd.read_csv("/Users/elbethelzewdie/Documents/customers-satisfaction-week2/customers-satisfaction-week2/data/review.csv")

Creating connection with the PostgreSQL Database

In [12]:
# -------------------------------------------------------
# Connect to PostgreSQL database
# -------------------------------------------------------

# Establish a connection to PostgreSQL using psycopg2
# You must make sure the database 'restaurant_reviews' is already created
conn = psycopg2.connect(
    host="localhost",              # PostgreSQL is running on your machine
    database="bank_reviews", # The database name you created
    user="postgres",                  # Your PostgreSQL username
    password="Admin"             # Your PostgreSQL password
)

# Create a cursor object — used to execute SQL commands
cur = conn.cursor()

Creating Tables

In [4]:
# -------------------------------------------------------
# CREATE TABLES
# -------------------------------------------------------

create_banks_table = """
CREATE TABLE IF NOT EXISTS banks (
    bank_id VARCHAR(200) PRIMARY KEY,
    bank_name VARCHAR(100),
    app_name VARCHAR(150)
);
"""

create_reviews_table = """
CREATE TABLE IF NOT EXISTS reviews (
    review_id SERIAL PRIMARY KEY,
    bank_id VARCHAR(200) REFERENCES banks(bank_id),
    review_text TEXT,
    rating INT,
    review_date DATE,
    sentiment_label VARCHAR(20),
    sentiment_score FLOAT,
    source VARCHAR(50)
);
"""

cur.execute(create_banks_table)
cur.execute(create_reviews_table)
conn.commit()


Insert into Banks Table

In [5]:
# -------------------------------------------------------
# INSERT INTO BANKS TABLE
# -------------------------------------------------------

for _, row in df_banks.iterrows():
    cur.execute(
        """
        INSERT INTO banks (bank_id, bank_name, app_name)
        VALUES (%s, %s, %s)
        ON CONFLICT (bank_id) DO NOTHING;
        """,
        (
            row["bank_id"],
            row["bank_name"],
            row["app_name"]
        )
    )
conn.commit()

Insert into Review table

In [8]:
# -------------------------------------------------------
# INSERT INTO REVIEWS TABLE
# -------------------------------------------------------

for _, row in df_reviews.iterrows():
    cur.execute(
        """
        INSERT INTO reviews (
            bank_id, review_text, rating, review_date,
            sentiment_label, sentiment_score, source
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s);
        """,
        (
            row["bank_id"],
            row["review_text"],
            row["rating"],
            row["review_date"],
            row["sentiment_label"],
            row["sentiment_score"],
            row["source"]
        )
    )
conn.commit()

Connection Closing

In [21]:
# -------------------------------------------------------
# CLOSE CONNECTION
# -------------------------------------------------------

cur.close()
conn.close()

print("✅ All tables created and data successfully inserted!")

✅ All tables created and data successfully inserted!


### Count total rows in each table

In [13]:
check_data_sql = """
SELECT 
    (SELECT COUNT(*) FROM banks) AS total_banks,
    (SELECT COUNT(*) FROM reviews) AS total_reviews,
    (SELECT AVG(rating) FROM reviews) AS avg_rating
;
"""

cur.execute(check_data_sql)
result = cur.fetchone()
print(result)

(3, 1623, Decimal('3.6124460874922982'))


### Count reviews per bank

In [14]:
reviews_per_bank_sql = """
SELECT b.bank_name, COUNT(r.review_id) AS total_reviews
FROM banks b
LEFT JOIN reviews r ON b.bank_id = r.bank_id
GROUP BY b.bank_name
ORDER BY total_reviews DESC;
"""


cur.execute(reviews_per_bank_sql)
result = cur.fetchall()
for row in result:
    print(row)

('Bank of Abyssinia', 574)
('Dashen Bank', 530)
('Commercial Bank of Ethiopia', 519)


### Average rating per bank

In [16]:
avg_rating_per_bank_sql = """
SELECT b.bank_name, AVG(r.rating) AS avg_rating
FROM banks b
LEFT JOIN reviews r ON b.bank_id = r.bank_id
GROUP BY b.bank_name;
"""


cur.execute(avg_rating_per_bank_sql)
result = cur.fetchall()
for row in result:
    print(row)

('Commercial Bank of Ethiopia', Decimal('3.9672447013487476'))
('Dashen Bank', Decimal('4.1283018867924528'))
('Bank of Abyssinia', Decimal('2.8153310104529617'))


### Count positive/neutral/negative sentiment per bank

In [17]:
sentiment_per_bank_sql = """
SELECT b.bank_name, r.sentiment_label, COUNT(*) AS count
FROM banks b
JOIN reviews r ON b.bank_id = r.bank_id
GROUP BY b.bank_name, r.sentiment_label
ORDER BY b.bank_name;
"""

cur.execute(sentiment_per_bank_sql)
result = cur.fetchall()
for row in result:
    print(row)

('Bank of Abyssinia', 'NEUTRAL', 5)
('Bank of Abyssinia', 'NEGATIVE', 317)
('Bank of Abyssinia', 'POSITIVE', 252)
('Commercial Bank of Ethiopia', 'NEGATIVE', 178)
('Commercial Bank of Ethiopia', 'NEUTRAL', 6)
('Commercial Bank of Ethiopia', 'POSITIVE', 335)
('Dashen Bank', 'POSITIVE', 373)
('Dashen Bank', 'NEGATIVE', 151)
('Dashen Bank', 'NEUTRAL', 6)


### Number of reviews by rating (1–5 stars)

In [18]:
count_reviews_by_rating_sql = """
SELECT 
    rating,
    COUNT(*) AS review_count
FROM reviews
GROUP BY rating
ORDER BY rating;
"""

cur.execute(count_reviews_by_rating_sql)
result = cur.fetchall()
for row in result:
    print(row)

(1, 435)
(2, 76)
(3, 91)
(4, 102)
(5, 919)


### Check for missing data (NULL checking)

In [19]:
check_missing_data_sql = """
SELECT
    SUM(CASE WHEN review_text IS NULL THEN 1 ELSE 0 END) AS missing_review_text,
    SUM(CASE WHEN rating IS NULL THEN 1 ELSE 0 END) AS missing_rating,
    SUM(CASE WHEN bank_id IS NULL THEN 1 ELSE 0 END) AS missing_bank_id,
    SUM(CASE WHEN review_date IS NULL THEN 1 ELSE 0 END) AS missing_review_date,
    SUM(CASE WHEN sentiment_label IS NULL THEN 1 ELSE 0 END) AS missing_sentiment_label,
    SUM(CASE WHEN sentiment_score IS NULL THEN 1 ELSE 0 END) AS missing_sentiment_score,
    SUM(CASE WHEN source IS NULL THEN 1 ELSE 0 END) AS missing_source
FROM reviews;
"""

cur.execute(check_missing_data_sql)
result = cur.fetchone()
print(result)

(0, 0, 0, 0, 0, 0, 0)


### Find banks that have no reviews (data consistency check)

In [20]:
banks_without_reviews_sql = """
SELECT 
    b.bank_id, 
    b.bank_name
FROM banks b
LEFT JOIN reviews r ON b.bank_id = r.bank_id
WHERE r.bank_id IS NULL;
"""

# Execute the SQL
cur.execute(banks_without_reviews_sql)

# Fetch results into a pandas DataFrame
df_banks_without_reviews = pd.DataFrame(cur.fetchall(), columns=['bank_id', 'bank_name'])

print("Banks with no reviews:")
print(df_banks_without_reviews)

Banks with no reviews:
Empty DataFrame
Columns: [bank_id, bank_name]
Index: []
